In [9]:
%pip install langchain-community langchain-core
%pip install sklearn
%pip install pandas
%pip install nltk


You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import json
import rdflib
import random
from typing import List
from langchain_community.graphs import OntotextGraphDBGraph
from constants import (
    TRAINED_MODEL_ID_PATH,
    TEST_DATA_PATH,
    SPARQL_FIX_PROMPT,
    QA_PROMPT,
    SPARQL_GENERATION_PROMPT,
)
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from tqdm import tqdm

In [11]:
class CustomOntotextGraphDBGraph(OntotextGraphDBGraph):
    def query(
        self,
        query: str,
    ) -> List[rdflib.query.ResultRow]:
        """
        Query the graph.
        """
        from rdflib.query import ResultRow

        res = self.graph.query(query)
        if res.type == "ASK":
            return [r for r in res if isinstance(r, bool)]
        return [r for r in res if isinstance(r, ResultRow)]

In [12]:
config = {
    "query_endpoint": "http://localhost:7200/repositories/imkg",
    "local_file": "/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/KG/iMKG.ttl",
    "local_file_format": "turtle",
}
try:
    graph = CustomOntotextGraphDBGraph(**config)
    print("Graph object created successfully.")
except Exception as e:
    print("Failed to create graph object:", e)

Graph object created successfully.


In [24]:
# Define the path to the test data
TEST_DATA_PATH = os.path.join(os.getcwd(), "data", "qa_test.json")
TEST_DATA_RESULTS_PATH = os.path.join(os.getcwd(), "data", "qa_test_results.json")

# Load the dataset
with open(TEST_DATA_PATH, "r", encoding="utf-8") as f:
    test_dataset = json.load(f)

# Select 10 random test data points
test_data_points = random.sample(test_dataset, 10000)

In [25]:
import requests

In [27]:
def get_generated_query(question: str):
    response = requests.post(
        "http://localhost:8000/query",
        json={"question": question},
    )
    response_json = response.json()
    return response_json["response"]


def compare_two_query(correct_query: str, generated_query: str):
    generated_results = set(graph.query(generated_query))
    sample_results = set(graph.query(correct_query))
    print(generated_results)
    print(sample_results)

    # Compare the results
    if generated_results == sample_results:
        return True
    return False


for index, data_point in enumerate(test_data_points):
    generated_query = get_generated_query(question=data_point["question"])
    if not generated_query:
        print(f"Generated query is invalid {generated_query}")
        continue
    data_point["generated_query"] = generated_query
    data_point["is_generated_query_correct"] = compare_two_query(
        correct_query=data_point["sparql"], generated_query=generated_query
    )
    print(f"Question: {index + 1} / {len(test_data_points)}\n{data_point}")

{(rdflib.term.Literal('Being Julia'), rdflib.term.URIRef('https://www.wikidata.org/entity/Q814790'))}
{(rdflib.term.Literal('Being Julia'), rdflib.term.URIRef('https://www.wikidata.org/entity/Q814790'))}
Question: 1 / 10000
{'qid': 'mcqa-9284b2c7-e67e-11ee-b6c8-58961d663d9c', 'question': 'do any movies have the same screenwriter as [The Letter] and have the tag [annette bening]', 'question_type': 'movie_to_writer_to_movie_constraint_tag', 'topic_entity': ['The Letter', 'annette bening'], 'topic_entity_id': ['https://www.wikidata.org/entity/Q104869439', 'https://www.wikidata.org/entity/Q106399271', 'https://www.wikidata.org/entity/Q1168233', 'https://www.wikidata.org/entity/Q1579140', 'https://www.wikidata.org/entity/Q21869523', 'https://www.wikidata.org/entity/Q2249047', 'https://www.wikidata.org/entity/Q3987983', 'https://www.wikidata.org/entity/Q6663542', 'https://www.wikidata.org/entity/Q7746987', 'https://www.wikidata.org/entity/Q7746988', 'https://www.wikidata.org/entity/MT1351'],

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [20]:
with open(TEST_DATA_RESULTS_PATH, "w", encoding="utf-8") as f:
    json.dump(test_data_points, f)